(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

# Pre-process the data

In [1]:
import csv
import pandas as pd
import numpy as np
import json

STOPWORDS_FD = 'stopwords'
STOPWORDS = {}
from collections import defaultdict

# From Homework 2
def get_words(filename):
    words = {}
    for line in open(filename):
        word = line.rstrip()
        words[word] = word
    return words

def readJson(f):
    for l in open(f):
        yield eval(l)

In [42]:
arr = []
for l in readJson('train.json'):
    tmp = [l['reviewerID'],l['asin'],l['overall']]
    arr.append(tmp)

df = pd.DataFrame(arr)
df.to_csv('reviewerID-asin.csv')

In [3]:
df = pd.read_csv('reviewerID-asin.csv')
users = []
items = []
for index,item in df.iterrows():
    s = str(item[1])
    i = str(item[2])
    users.append(s)
    items.append(i)
    
users = list(set(users))
items = list(set(items))

df1 = pd.DataFrame(users)
df2 = pd.DataFrame(items)
df1.to_csv('train_users.csv')
df2.to_csv('train_items.csv')

In [68]:
train_users = pd.read_csv('train_users.csv')
train_items = pd.read_csv('train_items.csv')
overall_rating = pd.read_csv('reviewerID-asin.csv')

user_avg_rating = {}
train_users.columns = ['index','user']
train_items.columns = ['index','item']

overall_rating.columns = ['index','user_id', 'item', 'rating']

group_by_user_avg_rating = overall_rating.groupby(['user_id'])['rating'].mean()

test = pd.DataFrame(group_by_user_avg_rating)

item_total = 0
item_count = 0
user_total = 0
user_count = 0

user_avg_rating = {}# average rating of a user has given
for index, item in test.iterrows():
    user_count = user_count + 1
    _id = str(index)
    rating = item
    user_total = user_total + float(rating)
    user_avg_rating[_id] = float(rating)

group_by_item_avg_rating = overall_rating.groupby(['item'])['rating'].mean()
test2 = pd.DataFrame(group_by_item_avg_rating)
test2.to_csv('item_avg_rating.csv')

item_avg_rating = {}# average rating of each item
for index, item in test2.iterrows():
    item_count = item_count + 1
    item_id = str(index)[4:]
    rating = float(item)
    item_avg_rating[item_id] = rating
    item_total = item_total + rating
    
#all_item_avg_score = 3.789
all_user_avg_score = 3.78
    

In [71]:
test = []
for l in open("pairs_Rating.txt"):
    u,i = l.strip().split('-')
    test.append(i)

test_set = []
for index, item in test2.iterrows():
    item_id = str(index)
    test_set.append(item_id)

test_set = list(set(test_set))
test = list(set(test))

all_existing_items = pd.DataFrame(test_set)
all_existing_items.columns = ['item']

allItems = pd.DataFrame.from_csv("test.csv")
allItems.columns = ['item','types','price']
allItems['item'] = allItems['item'].astype(str)

result = all_existing_items.merge(allItems, left_on='item', right_on='item', how='inner')
result.to_csv('all_existing_items_information.csv')

In [75]:
item_info = pd.read_csv('all_existing_items_information.csv')
item_info.columns = ['index','item','types','price']
cate_price = item_info.groupby(['types'])['price'].mean()
cate_price.to_csv('cate_price.csv')

item_price = item_info.groupby(['item'])['price'].mean()
item_price.to_csv('item_price.csv')

item_info = pd.read_csv('all_existing_items_information.csv')
item_info.columns = ['index','item','types','price']
cate_price = pd.read_csv('cate_price.csv')
cate_price.columns = ['types', 'prices']

item_cate_price = item_info.merge(cate_price, left_on='types', right_on='types', how='inner')
item_cate_price.to_csv('item_cate_price.csv')

# 1. Rating Prediction

In [41]:
import csv
import pandas as pd
import json
import numpy as np
from sklearn import linear_model

reviewerID_asin = pd.read_csv('reviewerID-asin.csv')
reviewerID_asin.columns = ['index','user','item','rate']

item_avg_rating = pd.read_csv('item_avg_rating.csv')
item_avg_rating.columns = ['item','avg_rate']

item_price = pd.read_csv('item_price.csv')
item_price.columns = ['item','avg_price']

user_rate = pd.read_csv('group_by_user_avg_rating.csv')
user_rate.columns = ['user','give_rate']

item_cate_price = pd.read_csv('item_cate_price.csv')
#item_cate_price.columns = ['item','cat','cat_price']
item_cate_price.drop(['price', 'index'], axis=1, inplace=True)

item_cate_price.columns = ['index', 'item','cat','cat_price']

tmp1 = reviewerID_asin.merge(item_avg_rating, left_on='item', right_on='item', how='inner')
tmp2 = tmp1.merge(item_price, left_on='item', right_on='item', how='inner')
tmp3 = tmp2.merge(user_rate, left_on='user', right_on='user', how='inner')
tmp4 = tmp3.merge(item_cate_price, left_on='item', right_on='item', how='inner')
tmp4.drop(['index_y', 'index_x'], axis=1, inplace=True)

X = tmp4.copy()
X.drop(['user', 'item','cat','rate'], axis=1, inplace=True)

y = tmp4.copy()
y.drop(['user', 'item','cat','avg_price','give_rate', 'cat_price', 'avg_rate'], axis=1, inplace=True)


train_X = []
for index, item in X.iterrows():
    tmp = [round(float(item[0]),2), float(item[1]), float(item[2]), float(item[3])]
    train_X.append(tmp)

train_Y = []
for index, item in y.iterrows():
    train_Y.append(float(item[0]))

# Create linear regression object
regr = linear_model.LinearRegression()
my_x = np.array(train_X)
my_y = np.array(train_Y)
# Train the model using the training sets
regr.fit(my_x, my_y)

# The coefficients
print('Coefficients: \n', regr.coef_)
#avg_rate , avg_price, give_rate, cat_price

testing = []
for l in open("pairs_Rating.txt"):
    u,i = l.strip().split('-')
    tmp = [u,i]
    testing.append(tmp)
    
user_item = pd.DataFrame(testing)
user_item = user_item[1:]
user_item.columns = ['user', 'item']

tp1 = user_item.merge(item_avg_rating, left_on='item', right_on='item', how='left')
tp2 = tp1.merge(item_price, left_on='item', right_on='item', how='left')
tp3 = tp2.merge(user_rate, left_on='user', right_on='user', how='left')
tp4 = tp3.merge(item_cate_price, left_on='item', right_on='item', how='left')
tp4.drop(['user', 'item','cat','index'], axis=1, inplace=True)

tp4.fillna(3.8, inplace=True)
testing_x = []
for index, item in tp4.iterrows():
    tmp = [round(float(item[0]),2), float(item[1]), float(item[2]), float(item[3])]
    testing_x.append(tmp)
    
h = regr.predict(testing_x)

print len(h)

predictions = open("predictions_Rating.txt", 'w')
count = 0
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    predictions.write(u + '-' + i + ',' + str(round(h[count],2)) + '\n')
    count = count + 1
predictions.close()



('Coefficients: \n', array([  4.41506014e-01,   2.11303382e-05,   8.80755253e-01,
         1.07855819e-04]))
100000


# 2.Purchase prediction



In [7]:
import csv
import pandas as pd
import json
import numpy as np
from sklearn import linear_model
reviewerID_asin = pd.read_csv('reviewerID-asin.csv')
reviewerID_asin.columns = ['index','user', 'item', 'score']
purchase_times_user = reviewerID_asin.groupby(['user'])['user'].count()
purchase_times_user.to_csv('purchase_times_user.csv')

purchase_times_item = reviewerID_asin.groupby(['item'])['item'].count()
purchase_times_item.to_csv('purchase_times_item.csv')



In [13]:
import csv
import pandas as pd
import json
import numpy as np
from sklearn import linear_model



reviewerID_asin = pd.read_csv('reviewerID-asin.csv')
reviewerID_asin.columns = ['index','user', 'item', 'score']

item_avg_rating = pd.read_csv('item_avg_rating.csv')
item_avg_rating.columns = ['item','avg_rate']

item_price = pd.read_csv('item_price.csv')
item_price.columns = ['item','avg_price']

user_rate = pd.read_csv('group_by_user_avg_rating.csv')
user_rate.columns = ['user','give_rate']

purchase_times_item = pd.read_csv('purchase_times_item.csv')
purchase_times_item.columns = ['item', 'item_times']

purchase_times_user = pd.read_csv('purchase_times_user.csv')
purchase_times_user.columns = ['user', 'user_times']

tmp1 = reviewerID_asin.merge(item_avg_rating, left_on='item', right_on='item', how='inner')
tmp2 = tmp1.merge(item_price, left_on='item', right_on='item', how='inner')
tmp3 = tmp2.merge(user_rate, left_on='user', right_on='user', how='inner')
tmp4 = tmp3.merge(purchase_times_user, left_on='user', right_on='user', how='inner')
tmp5 = tmp4.merge(purchase_times_item, left_on='item', right_on='item', how='inner')
tmp5.drop(['user', 'index', 'item', 'score'], axis=1, inplace=True)

train_x = tmp5.as_matrix()
length = len(train_x)
train_y = np.random.choice([0, 1], size=(length,), p=[0.5/10, 9.5/10])

h = .02  # step size in the mesh
logreg = linear_model.LogisticRegression(C=1e5)

logreg.fit(train_x, train_y)

testing = []
for l in open("pairs_Rating.txt"):
    u,i = l.strip().split('-')
    tmp = [u,i]
    testing.append(tmp)
    
user_item = pd.DataFrame(testing)
user_item = user_item[1:]
user_item.columns = ['user', 'item']
print len(user_item)
tp1 = user_item.merge(item_avg_rating, left_on='item', right_on='item', how='left')
tp2 = tp1.merge(item_price, left_on='item', right_on='item', how='left')
tp3 = tp2.merge(user_rate, left_on='user', right_on='user', how='left')
tp4 = tp3.merge(purchase_times_user, left_on='user', right_on='user', how='left')
tp5 = tp4.merge(purchase_times_item, left_on='item', right_on='item', how='left')
tp5.give_rate.fillna(3.8, inplace=True)
tp5.item_times.fillna(5.84, inplace=True)
tp5.user_times.fillna(1.96, inplace=True)
tp5.drop(['user', 'item'], axis=1, inplace=True)
print len(tp5)
test_x = tp5.as_matrix()

c = 0
for test in tp5.iterrows():
    print test
    c = c + 1
    if c == 10000:
        break
'''
Z = logreg.predict(test_x)
c = 0
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    predictions.write(u + '-' + i + "," + Z(c) +"\n")
    c = c + 1
predictions.close()
'''
#score avg_rate avg_price give_rate 

100000
100000
(0, avg_rate       3.833333
avg_price     19.650000
give_rate      3.500000
user_times     8.000000
item_times     6.000000
Name: 0, dtype: float64)
(1, avg_rate      1.25
avg_price     0.00
give_rate     3.00
user_times    1.00
item_times    4.00
Name: 1, dtype: float64)
(2, avg_rate       3.433333
avg_price     29.990000
give_rate      5.000000
user_times     2.000000
item_times    60.000000
Name: 2, dtype: float64)
(3, avg_rate       3.433333
avg_price     29.990000
give_rate      3.800000
user_times     5.840000
item_times    60.000000
Name: 3, dtype: float64)
(4, avg_rate       3.433333
avg_price     29.990000
give_rate      3.800000
user_times     5.840000
item_times    60.000000
Name: 4, dtype: float64)
(5, avg_rate       3.433333
avg_price     29.990000
give_rate      3.000000
user_times     2.000000
item_times    60.000000
Name: 5, dtype: float64)
(6, avg_rate       3.433333
avg_price     29.990000
give_rate      5.000000
user_times     3.000000
item_times    60.

'\nZ = logreg.predict(test_x)\nc = 0\npredictions = open("predictions_Purchase.txt", \'w\')\nfor l in open("pairs_Purchase.txt"):\n    if l.startswith("reviewerID"):\n    #header\n        predictions.write(l)\n        continue\n    u,i = l.strip().split(\'-\')\n    predictions.write(u + \'-\' + i + "," + Z(c) +"\n")\n    c = c + 1\npredictions.close()\n'

In [14]:
purchase_times_user = pd.read_csv('purchase_times_user.csv')
purchase_times_user.columns = ['item', 'user_times']
m = purchase_times_user['user_times'].mean()
print m

1.96202496875


-----------------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()